### Chapter 7. Extending the OLG model
Reference: Fehr, H. and Kindermann, F. 2018. The overlapping generations model. Introduction to Computational Economics Using Fortran. Oxford University Press. pp253-286.

In [1]:
##### Program 7.2 The global variables for endogenous human capital

import numpy as np

# Parameters
"""
    T: transition path periods
    J: age
    JR: the year household retires
    γ: risk aversion
    egam: denominator of utility function
    β: discount factor
    ν: leisure preference parameter
    ρ: intratemporal elasticity of substitution between consumption and leisure
    eρ: denominator of leisure preference
    α: capital share
    δ: depreciation
    tol: tolerance
    damp: speed of convergence
    itermax: maximum iteration number
    ξ: parameter of education function ξe^υ
    υ: parameter of education function ξe^υ
    ϵ: the strength o the externality in Y=(K^α)L^(1-α)H^ϵ
    ----------------------------------------------------------------------------
    w: wage rate
    r: rate of capital return
    wn: after tax wage (net wage)
    Rn: after tax capital return
    p: consumption price
    K: aggregate capital
    L: aggregate labour
    A: aggregate assets
    C: aggregate consumption
    I: aggregate investment
    B: aggregate debt
    G: government spending
    k: capital per capita
    a: savings
    e: education investment
    c: consumption
    U: utility
    l: leisure
    h: household productivity (human capital)
    Y: aggregate output
    Ba: debt or assets of the agency per capita of workers
    Bf: future period debt B_{t+1}
    Tb: total debt (1+np)B_{t+1}+(1+r)B_t
    TXr: total tax revenue
    Tpen: total pension
    H: the average stock o human capitak per worker
    
"""

def initialize():

    global T, J, JR, γ, egam, β, α, δ, tol, damp, itermax, ξ, ϵ, υ, w, r, wn, Rn, p, K, \
    L, A, C, I, B, G, k, U, Y, Ba, Bf, h, Tb, TXr, Tpen, H, n_p, gy, by, κ, lsra_on, \
    smopec, τk, τc, τw, τr, τp, τs, tax, pen, a, e, c, v, m

    # Parameters
    T = 27; J = 3; JR = 3; γ = .5; egam = 1.0 - 1.0/γ; β = .9
    α = .3; δ = .0; tol = 1e-6; damp = 0.3
    itermax = 200; ξ = 2.0; ϵ = 0.0; υ = 0.5

    # Variables
    w  = np.zeros(T+1); r = np.zeros(T+1); wn = np.zeros(T+1)
    Rn = np.zeros(T+1); p = np.zeros(T+1); K  = np.zeros(T+1)
    L  = np.zeros(T+1); A = np.zeros(T+1); C  = np.zeros(T+1)
    I  = np.zeros(T+1); B = np.zeros(T+1); G  = np.zeros(T+1)
    k  = np.zeros(T+1); U = np.zeros((J,T+1)); Y = np.zeros(T+1)
    Ba = np.zeros(T+1); Bf = np.zeros(T+1); h = np.zeros((J,T+1))
    Tb = np.zeros(T+1); TXr = np.zeros(T+1); Tpen = np.zeros(T+1)
    H = np.ones(T+1)

    # Set model parameters
    n_p = np.zeros(T+1) + .2    #population growth rate
    gy  = 0.205                 #government spending proportion
    by  = np.zeros(T+1)         #debt proportion
    κ   = np.zeros(T+1)         #pension proportion
    lsra_on = False             #include welfare analysis or not
    smopec = False              #include small open economy or not

    # Initialize tax rates shadow wages and pensions
    τk  = np.zeros(T+1)         #investment tax rate
    τc = np.zeros(T+1)          #consumption tax rate
    τw = np.zeros(T+1)          #labour income tax rate
    τr = np.zeros(T+1)          #capital income tax rate
    τp = np.zeros(T+1)          #payroll tax rate
    τs = np.zeros(T+1)          #subsidy rate of the government for education costs
    tax = np.ones(T+1)          #tax balancing index
    pen = np.zeros((J,T+1))     #pension

    # Initialize assets, LSRA payments and debt holdings
    a = np.zeros((J,T+1))       #assets
    e = np.zeros(T+1)           #education investment
    c = np.zeros((J,T+1))       #consumption
    v = np.zeros(T+J-1)         #reqiured transfer
    h[0,:] = 1.0                #initial productivity
    h[1:J,:] = .0               #initialize later productivity

    # Size of cohorts in specific year
    m = np.zeros((J,T+1))       #population parameter
    for t in range(T+1):
        m[0,t] = 1.0            #in period 0, m=1
        tm = year(t, 1, 0)      #time index t-1
        for j in range(1,J):
            m[j,t] = m[j-1,tm]/(1.0 + n_p[t]) #m_{j,t}=m_{j-1,t-1}/(1+np)
    return None

In [2]:
## calculates year at which age j agent is jp
def year(t, j, jp):
    """This function defines the year index"""
    year = t + jp - j  #year i age j period ij (j-1, j+1, j+2)
    if t == 0 or year<=0:
        year = 0       #initial period
    if t == T or year>=T:
        year = T       #last period
    return year

In [3]:
## Function: factor price:
def factor_prices(t):
    """This function defines multidimensional factor price calculation for period i."""
    global k, r, w, wn, Rn, p
    k[t]   = K[t]/L[t]                  #per capita capital stock
    if smopec and t > 0:
        r[t] = r[0]
    else:
        r[t] = (1.0 - τk[t])*(α*k[t]**(α-1.0)*H[t]**ϵ-δ)  #(1-τk)*r
  
    w[t]   = (1.0 - α)*k[t]**α*H[t]**ϵ  #wage rate

    wn[t]  = w[t]*(1.0 - τw[t] - τp[t]) #the net wage of a worker in period t
        
    Rn[t]  = 1.0 + r[t]*(1.0 - τr[t])   #net interest
    p[t]   = 1.0 + τc[t]                #consumer price
    return None

In [4]:
## present value of resources for household aged j in year t

def get_W(j, t):
    """This function calculates the total income at each age j at period t."""
    # get current value of resources
    Assets = wn[t]*h[j,t] + pen[j,t]       #initial total income include the wage and pension
    
    if t == 1 and j > 0:              #if period 1 and age 1,2,...
        ## calful about index!!!
        Assets = Assets + Rn[t] * a[j,t] + v[J - j - 1] #initial assets + capital return + required transfer 
    if j ==0:                         #if age 0
        Assets = Assets + v[t+J-2] - (1.0 -τs[t])*e[t]*h[0,t]*wn[t]    
        #initial assets + required transfer - education expense
        
    # Iterate over remainder of life span
    PRn = 1.0
    for jp in range(j+1,J):  #loop for remainder of life span
        tp = year(t,j,jp)    #time index for remainder of life span
        PRn = PRn*Rn[tp]     #cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        Assets = Assets + (wn[tp]*h[jp,tp] + pen[jp,tp])/PRn   #W equation above (7.18)
    return Assets

## marginal consumption for household aged j in year t

def get_Psi(j,t):
    """This function calculates the consumption proportion at each age j at period t."""
    Psi = 1.0         #set initial value of Psi
    PRn = 1.0         #PRn in the period 0 is 1
    
    for jp in range(j+1,J):  #loop for each future ages
        tp = year(t, j ,jp)  #year interval t+jp-j
        PRn = PRn*Rn[tp]     #cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        Psi = Psi + β**((jp-j)*γ)*(p[tp]/(PRn*p[t]))**(1.0-γ)  #part inside big bracket of (7.16)
    Psi = 1.0/(p[t]*Psi)     #v/p/(big bracket) equation (7.16) with v=1 since leisure preference ν=0
    
    return Psi

In [5]:
## Calculating the optimal consumption path

def get_path(j,t):
    """This function calculates the optimal consumption path at each age j at period t."""
    
    global c, a
    
    PRn = 1.0         #PRn in the period 0 is 1

    for jp in range(j+1,J):  #loop for future ages (j+1,...,J)
        # get future and previous year as well as interest factor
        tp = year(t,j,jp)    #t+jp-j
        tm = year(t,j,jp-1)  #t+jp-j-1
        PRn = PRn*Rn[tp]     #cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        
        # get consumption and leisure
        c[jp,tp] = (β**(jp-j)*PRn*p[t]/p[tp])**γ*c[j,t]      #c_{2,t+1} and c_{3,t+2} equation (7.13&14)
            
        # calculate assets 
        a[jp,tp] = wn[tm]*h[jp-1,tm] + pen[jp-1,tm] + Rn[tm]*a[jp-1,tm] -p[tm]*c[jp-1,tm]  
        #savings as all income (labour income + pension + capital income) - consumption

        if tp == 2:                         #t=2
            a[jp,tp] = a[jp,tp] + v[J-jp]   #a + required transfer
        if tp > 2 and jp == 1:              #t>2, age 1
            a[jp,tp] = a[jp,tp] + v[tm+J-2] #a + required transfer
        if jp == 1:                         #age 1
            a[jp,tp] = a[jp,tp] - (1.0 - τs[tm])*e[tm]*wn[tm] #a - education investment
            
    return None


In [6]:
## Calculating individual decisions in a certain year

def decisions(t):
    """This function defines the decisions of consumption and savings in each period t."""
    global c, e, h
    
    tp = year(t, 0, 1)               #t+1
    e[t] = (ξ*υ*wn[tp]/(wn[t]*(1.0-τs[t])*Rn[tp]))**(1.0/(1.0-υ)) #equation (7.19)
    h[1,tp] = 1.0 + ξ*e[t]**υ        #equation (7.17) h_{2,t+1}/h_1=1+ξe^υ, h_1=1
    
    c[0,t] = get_Psi(0,t)*get_W(0,t) #c=ΨW
    
    get_path(0,t)                    #calculate consumption path at age 0 period t
    
    # derive behaviour for all other cohorts in year 1 of transition
    if t == 1:                       #at t=1
        for j in range(1,J):         #for each age
            c[j,t] = get_Psi(j,t)*get_W(j,t)    #c=ψW
            get_path(j,t)                       #calculate consumption path at age j period t
    return None


In [7]:
## calculating quantities in a certain year

def quantities(t):
    """This function defines the aggregated quantities of C, A, H, Y, I, G, B, K in period t."""
    global G, C, A, L, H, Y, B, K, Bf, Tb, I
    
    tm = year(t,1,0)   #time index t-1
    tp = year(t,0,1)   #time index t+1
    
    if t == 0:         #initial goverment spending
        G[t] = gy*Y[t] #goverment spending
    else:              #other periods
        G[t] = G[0]    #same as period 0
        
    # aggregate individual decisions
    C[t] = .0          #initial value of aggregate consumption
    A[t] = .0          #initial value of aggregate assets
    L[t] = .0          #initial value of aggregate labour input
    H[t] = .0          #initial value of aggregate human capital stock (productivity)
    
    for j in range(J): #for all ages
        C[t] = C[t] + c[j,t]*m[j,t] #cumulated aggregate consumption with population parameters mt=m_{t-1}/(1+np)
        A[t] = A[t] + a[j,t]*m[j,t] #cumulated aggregate assets with population parameters mt=m_{t-1}/(1+np)
        L[t] = L[t] + h[j,t]*m[j,t] #cumulated aggregate labour input (h) with population parameters mt=m_{t-1}/(1+np)
        if j < JR-1:                #if not retired
            H[t] = H[t] + m[j,t]    #aggregate human capital stock is accumulating
    L[t] = L[t] - e[t]*h[0,t]       #L_t=(above L_t)-eh equation (7.20)
    H[t] = L[t]/(H[t] - e[t])       #H_t=L_t/(above H-e) equation (7.20)
    Y[t] = K[t]**α*L[t]**(1.0-α)*H[t]**ϵ    #aggregate output
    B[t] = by[tm]*Y[t]              #aggregate government debt
    
    # derive capital in small open or closed economy
    if smopec and t>0:
        K[t]  = L[t]*((r[t]*(1.0/(1.0-τk[t])) + δ)/(α*H[t]**ϵ))**(1.0/(α-1.0)) #converted from r=α(K/L)^(α-1)H^ϵ-δ 
        Bf[t] = A[t] - K[t] - Ba[t] - B[t]        #debt constraint
        Tb[t] = (1.0+n_p[tp])*Bf[tp] - (1.0+r[t])*Bf[t]  #total debt
    else:
        K[t] = damp*(A[t]-B[t]-Ba[t]) + (1.0-damp)*K[t]  #converging capital stock
    
    I[t] = (1.0+n_p[tp])*K[tp] - (1.0 - δ)*K[t]   #aggregate investment
    
    return None


In [8]:
# calculating government parameters

def government(t):
    """This function defines the government parameters and tax revenues at period t."""
    global τc, τk, τw, τr, pen, Tpen, τp, TXr
    tp = year(t, 0, 1)             #t+1
    
    taxrev = np.zeros(5)           #set 5 types of taxes
    
    taxrev[0] = τc[t]*C[t]         #total consumption tax
    taxrev[1] = τw[t]*w[t]*L[t]    #total labour income tax
    taxrev[2] = τr[t]*r[t]*A[t]    #total capital income tax (on interest return)
    taxrev[3] = τk[t]*(Y[t] - w[t]*L[t]-(δ)*K[t])  #total savings or investment tax
    taxrev[4] = τs[t]*e[t]*wn[t]   #total education subsidy
    
   
    
    # get budget balancing tax rate
    if tax[t] == 1:      #(government surplus-τcC)/C
        τc[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[1] + taxrev[2] + \
                    taxrev[3]+(1.0 + n_p[tp])*B[tp]))/C[t]   
    elif tax[t] == 2:    #[government surplus-τw(wL)-τr(rA)]/(wL+rA) assuming same labour and capital income tax
        τw[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[3] + \
                    (1.0+n_p[tp])*B[tp]))/(w[t]*L[t] + r[t]*A[t])
        τr[t] = τw[t]    #same labour and capital income tax
    elif tax[t] == 3:    #[government surplus-τw(WL)]/(wL)
        τw[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[2] + \
                    taxrev[3] + (1.0 + n_p[tp])*B[tp]))/(w[t]*L[t])
    else:                #[government surplus-τr(rA)]/(rA)
        τr[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[1] + \
                    taxrev[3]+ (1.0 + n_p[tp])*B[tp]))/(r[t]*A[t])
    
    TXr[t] = np.sum(taxrev[0:4]) - taxrev[4]
    
    # get budget balancing social security contribution
    pen[JR-1:J,t] = κ[t]*w[t] #pension at each period
    Tpen[t] = .0              #initial total pension
    for j in range(JR-1,J):   #for retired periods
        Tpen[t] = Tpen[t] + pen[j,t]*m[j,t] #total pension with population parameters mt=m_{t-1}/(1+np)
        
    τp[t] = Tpen[t]/(w[t]*L[t]) #payroll tax rate
    return None

In [9]:
## Compute household utility

def utility(t):
    """This function defines expected utility at each age for each period i in [0,T]."""
    global U
    
    # for first generation
    U[0,t] = .0
    for j in range(J):
        tp = year(t, 0, j)
        U[0,t] = U[0,t] + β**j*c[j,tp]**egam/egam
    
    # for current total population if year = 0 or 1
    if t < 2:
        for j in range(1,J):
            U[j,t] = .0
            for jp in range(j,J):
                tp = year(t, j, jp)
                U[j,t] = U[j,t] + β**(jp-j)*c[jp,tp]**egam/egam
                
    return None

In [10]:
## Calculating lsra transfers

def lsra():
    """This function calculates social welfare. True to include, False to exclude."""
    global Ba, v, ustar

    #calculate utility for each generation
    for t in range(1,T+1):
        utility(t)   #get utility for each period i=0,1,2,...T
        
    Ba = np.zeros(T+1)  #initialize debt
    for j in range(1,J):
        v[J-j-1] = v[J-j-1] + get_W(j,1)*((U[j,0]/U[j,1])**(1.0/egam)-1.0)
        #v: required transfer that bring older cohorts in the reform year from their current utility level 
        #back to their initial equilibrium utilities
        Ba[2] = Ba[2] + v[J-j-1]*m[j,1]  #debt or assets of the agency per capita of workers
    
    #Calculate the following separated terms of the equation to solve for U^*.
    # long run equilibrium (last period) 
    PV = v[T+J-2]*(1.0+r[T])/(r[T]-n_p[T])                                 #v/(r-n_p) in period T part
    sum1 = get_W(0,T)*(1.0+r[T])/(r[T]-n_p[T])                             #w/(r-n_p) in period T part
    sum2 = get_W(0,T)*(U[0,T]*egam)**(-1.0/egam)*(1.0+r[T])/(r[T]-n_p[T])  #normalized W/Ut in period T part
    
    # transition path (backward induction)
    for t in range(T-1,0,-1): # from T to 1.
        tp = year(t,0,1)      #t+1
        PV = PV*(1.0+n_p[tp])/(1.0+r[tp]) + v[t+J-2]                       #v(1+np)/(1+r) part
        sum1 = sum1*(1.0+n_p[tp])/(1.0+r[tp]) + get_W(0,t)                 #W(1+np)/(1+r) part
        sum2 = sum2*(1.0+n_p[tp])/(1.0+r[tp]) + get_W(0,t)*(U[0,t]*egam)**(-1.0/egam) # normalized W/Ut part
       
    # calculate ustar for future generations 
    ustar = ((sum1 - Ba[2] - PV)/sum2)**egam/egam   #the identical utility level for future generations
    
    # calculate transfers to future generations and debt of LSRA
    for t in range(1,T+1):
        v[t+J-2] = v[t+J-2] + get_W(0,t)*((ustar/U[0,t])**(1.0/egam)-1.0)  #lump-sum transfers for future cohorts
        if t == 2:
            Ba[2] = (Ba[2] + v[J-1])/(1.0 + n_p[2])               #B_{t+1}=(above Ba+v)/(1+np) equation on page 243
        if t > 2:
            Ba[t] = ((1.0+r[t-1])*Ba[t-1] + v[J+t-3])/(1.0+n_p[t])#B_{t+1}=((1+r)Ba+v)/(1+np) same equation on page 243
    return None

In [11]:
# solves inital steady state using Gauss-Seidel

def get_SteadyState():
    """This function calculates inital steady state using Gauss-Seidel."""
    K[0] = 1.0  #initial capital
    L[0] = 1.0  #initial labour
    
    for i in range(itermax+1):
        factor_prices(0)
        decisions(0)
        quantities(0)
        government(0)
        if abs(Y[0]-C[0]-I[0]-G[0])/Y[0] < tol:
            break
    utility(0)
    
    if i < itermax:
        print('Iteration: ', i, ' Diff: ',abs(Y[0]-C[0]-I[0]-G[0])/Y[0])
    else:
        print('!!! No equilibrium found !!!')
    
    return None

In [12]:
## sovles for transition path using Gauss-Seidel:

def get_Transition():
    """This function calculates each period economy and checks for market clearing conditions."""
    global a, K, L, HEV, H, h, Δ
    
    # Initialize values from initial equilibrium
    a[:,1] = a[:,0]   #saving
    h[1,1] = h[1,0]   #productivity
    K[:] = K[0]       #aggregate capital
    L[:] = L[0]       #aggregate labour
    H[:] = H[0]       #aggregate human capital
    
    for ite in range(itermax+2):
        # get prices, decisions and quantites
        for t in range(T+1):
            factor_prices(t)
        
        for t in range(T+1):
            decisions(t)
        
        if lsra_on:
            lsra()
        
        for t in range(T+1):
            quantities(t)
        
        for t in range(T+1):
            government(t)
            
        # check for the number of markets in equilibrium
        nmarket = 0
        for t in range(1,T+1):
            if  abs(Y[t]-C[t]-I[t]-G[t]-Tb[t])/Y[t] < tol:
                nmarket = nmarket + 1
        if nmarket == T:
            break
    if ite < itermax:
        print('Iteration: ',ite,' Markets: ',nmarket,' Diff: ',np.max(np.abs(Y-C-I-G-Tb)/Y))
    else:
        print('!!! No equilibrium found !!!')
    if lsra_on == True:
        Δ = (ustar/U[0,0])**(1/egam)-1    #HEV particularly under ustar
    
    for t in range(T+1):
        utility(t)
    HEV = np.zeros(T+2)
    for t in range(2,T+2):
        HEV[t] = ((U[0,t-1]/U[0,0])**(1/egam)-1)*100
    HEV[1] = ((U[1,1]/U[1,0])**(1/egam)-1)*100 # generation 0
    HEV[0] = ((U[2,1]/U[2,0])**(1/egam)-1)*100 # generation -1
    return None

In [13]:
## Table 7.4
initialize()
tax[1:] = 3 # Table 7.4
get_SteadyState()
get_Transition()
print('---------------------------------------------')
print('Table 7.4 From consumption to labour-income \ntaxation with variable labour supply')
print('---------------------------------------------')
print(' t   τw   τc    C    e    h2   L    K    w    r    HEV')
print(-1,'                                            ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
for t in [T]:
    print(' T','%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------')
initialize()
tax[1:] = 3 # Table 7.4
get_SteadyState()
lsra_on = True
get_Transition()
print(Δ)

Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  57  Markets:  27  Diff:  8.743007465655273e-07
---------------------------------------------
Table 7.4 From consumption to labour-income 
taxation with variable labour supply
---------------------------------------------
 t   τw   τc    C    e    h2   L    K    w    r    HEV
-1                                              27.96
 0 0.00 0.27 0.82 0.13 1.72 2.31 0.18 0.33 1.76 -2.66
 1 0.29 0.00 0.88 0.09 1.72 2.35 0.18 0.33 1.79 -10.85
 2 0.32 0.00 0.75 0.09 1.60 2.24 0.15 0.31 2.03 -16.93
 3 0.33 0.00 0.72 0.09 1.60 2.25 0.13 0.30 2.19 -19.75
 4 0.34 0.00 0.70 0.09 1.59 2.24 0.12 0.29 2.29 -21.55
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 T 0.35 0.00 0.67 0.08 1.58 2.23 0.11 0.29 2.44 -24.04
---------------------------------------------
Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  61  Markets:  27  Diff

In [14]:
## Table 7.5
initialize()
tax[2:] = 3 # Table 7.5
get_SteadyState()
get_Transition()
print('---------------------------------------------')
print('Table 7.5 From consumption to labour-income \ntaxation with policy announcement')
print('---------------------------------------------')
print(' t   τw   τc    C    e    h2   L    K    w    r    HEV')
print(-1,'                                            ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
for t in [T]:
    print(' T','%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------')
initialize()
tax[2:] = 3 # Table 7.5
lsra_on = True
get_SteadyState()
get_Transition()
print(HEV[T+1])

Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  57  Markets:  27  Diff:  8.743007465655273e-07
---------------------------------------------
Table 7.5 From consumption to labour-income 
taxation with policy announcement
---------------------------------------------
 t   τw   τc    C    e    h2   L    K    w    r    HEV
-1                                              1.04
 0 0.00 0.27 0.82 0.13 1.72 2.31 0.18 0.33 1.76 6.32
 1 0.00 0.27 0.82 0.08 1.72 2.36 0.18 0.33 1.79 -0.52
 2 0.30 0.00 0.86 0.10 1.56 2.20 0.20 0.34 1.63 -8.51
 3 0.31 0.00 0.79 0.09 1.62 2.26 0.16 0.31 1.94 -14.95
 4 0.33 0.00 0.73 0.09 1.60 2.24 0.14 0.30 2.13 -18.83
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 T 0.35 0.00 0.67 0.08 1.58 2.23 0.11 0.29 2.44 -24.04
---------------------------------------------
Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  33  Markets:  27  Diff:  0.08

In [15]:
## Table 7.6
initialize()
τs[1:] = .25 # Table 7.6
get_SteadyState()
get_Transition()
print('---------------------------------------------')
print('Table 7.6 Education subsidies without human-capital externalities')
print('---------------------------------------------')
print(' t   τs   τc    C    e   h2    L    K    w    r   HEV')
print(-1,'                                            ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
for t in [T]:
    print(' T','%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------')
initialize()
τs[1:] = .25 # Table 7.6
lsra_on = True
get_SteadyState()
get_Transition()
print(HEV[T+1])

Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  30  Markets:  27  Diff:  9.712278054163521e-07
---------------------------------------------
Table 7.6 Education subsidies without human-capital externalities
---------------------------------------------
 t   τs   τc    C    e   h2    L    K    w    r   HEV
-1                                              -3.30
 0 0.00 0.27 0.82 0.13 1.72 2.31 0.18 0.33 1.76 0.23
 1 0.25 0.29 0.81 0.20 1.72 2.23 0.18 0.33 1.73 1.50
 2 0.25 0.29 0.83 0.21 1.90 2.37 0.17 0.32 1.86 -0.16
 3 0.25 0.29 0.83 0.21 1.92 2.39 0.17 0.32 1.88 -0.49
 4 0.25 0.29 0.83 0.21 1.92 2.39 0.17 0.32 1.90 -0.85
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 T 0.25 0.29 0.82 0.21 1.92 2.39 0.17 0.32 1.91 -1.12
---------------------------------------------
Iteration:  44  Diff:  9.598363679568825e-07
Iteration:  45  Markets:  27  Diff:  0.010203613411862196


In [16]:
## Table 7.7
initialize()
ϵ = .2;τs[1:] = .25 # Table 7.7
get_SteadyState()
get_Transition()
print('---------------------------------------------')
print('Table 7.7 Education subsidies with human-capital externalities')
print('---------------------------------------------')
print(' t   τs   τc    C    e   h2    L    K    w    r   HEV')
print(-1,'                                            ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
print(' .   .    .    .    .    .    .    .    .    .    . ')
for t in [T]:
    print(' T','%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%h[1,t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------')
initialize()
lsra_on = True
ϵ = .2;τs[1:] = .25 # Table 7.7
get_SteadyState()
get_Transition()
print(Δ)

!!! No equilibrium found !!!
!!! No equilibrium found !!!
---------------------------------------------
Table 7.7 Education subsidies with human-capital externalities
---------------------------------------------
 t   τs   τc    C    e   h2    L    K    w    r   HEV
-1                                              -3.49
 0 0.00 0.28 0.87 0.14 1.74 2.32 0.20 0.36 1.68 0.70
 1 0.25 0.30 0.86 0.22 1.74 2.24 0.20 0.36 1.64 2.38
 2 0.25 0.29 0.89 0.23 1.93 2.38 0.19 0.35 1.79 1.55
 3 0.25 0.29 0.90 0.22 1.95 2.40 0.19 0.35 1.80 1.60
 4 0.25 0.29 0.90 0.23 1.95 2.40 0.18 0.35 1.81 1.44
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 .   .    .    .    .    .    .    .    .    .    . 
 T 0.25 0.29 0.90 0.23 1.95 2.40 0.18 0.35 1.81 1.35
---------------------------------------------
!!! No equilibrium found !!!
!!! No equilibrium found !!!
-0.004063661748050618
